# Q2 — State-level accident rates per population / per VMT

Which locations or states experience the highest accident frequency per population or per vehicle miles?   
Goal: compute accidents per 100k population and per vehicle miles traveled (VMT).

# Imports and loads

In [1]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
import matplotlib.pyplot as plt

DATA_PATH = "../Data/us_accidents_sample_500k_clean.csv"
df = pd.read_csv(DATA_PATH, parse_dates=["Start_Time"], low_memory=False)
df = df.dropna(subset=["State"])


# Aggregate by state

In [2]:
state_counts = df.groupby("State").size().rename("crashes").reset_index()
state_counts = state_counts.sort_values("crashes", ascending=False)
state_counts.head()

,State,crashes
3,CA,100935
8,FL,49038
41,TX,35339
38,SC,22870
25,NC,20257


## 2) Merge population/VMT data
Need external population data (Census) or VMT (FHWA).  

- Expected columns in `state_pop.csv`: `state_code` (e.g., 'CA'), `population`, `vmt` (annual vehicle miles traveled)
